### Porovnání kvůli nalezení nejvhodnějších labelů

In [6]:
import sys
sys.path.append('../')
from functions.functions import count_SIR, STFT1024, get_signal_and_interference, calculate_SIR_for_frames
from functions.iFICA import iFICA

import numpy as np
import soundfile as sf
import pandas as pd

import plotly.graph_objects as go

In [3]:
number_of_samples = 2000

SIR_pilot1 = np.zeros((number_of_samples, 1))
SIR_pilot2 = np.zeros((number_of_samples, 1))
SIR_pilot3 = np.zeros((number_of_samples, 1))
SIR_pilot4 = np.zeros((number_of_samples, 1))

In [3]:
def get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot):
    [w, _, _, _] = iFICA(X, np.ones((nr_of_mics,1,K)), pilot);
    x_soi, x_int = get_signal_and_interference(w, SOI, INT)
    SIR_final, _ = count_SIR(x_soi, x_int)
    return SIR_final

In [4]:
def create_pilot1(sir_values):
    pilot = np.zeros_like(sir_values)
    
    pilot[(sir_values >= 0.0)] = 1.0
    pilot[sir_values < 0.0] = 0.0
    
    return pilot

In [5]:
def create_pilot2(sir_values):
    pilot = np.zeros_like(sir_values)
    
    pilot[(sir_values >= 1.0)] = 1.0
    pilot[(sir_values >= 0.0) & (sir_values < 1.0)] = 0.5
    pilot[sir_values < 0.0] = 0.0
    
    return pilot

In [6]:
def create_pilot3(sir_values):
    pilot = np.zeros_like(sir_values)
    
    pilot[sir_values >= 4.5] = 1.0
    pilot[(sir_values >= 3.0) & (sir_values < 4.5)] = 0.75
    pilot[(sir_values >= 1.5) & (sir_values < 3.0)] = 0.5
    pilot[(sir_values >= 0.0) & (sir_values < 1.5)] = 0.25
    pilot[sir_values < 0.0] = 0.0
    
    return pilot

In [7]:
def create_pilot4(sir_values):
    pilot = np.zeros_like(sir_values)
    
    pilot[sir_values >= 3.0] = 1.0
    pilot[(sir_values >= 2.4) & (sir_values < 3.0)] = 0.75
    pilot[(sir_values >= 1.8) & (sir_values < 2.4)] = 0.5
    pilot[(sir_values >= 0.0) & (sir_values < 1.8)] = 0.25
    pilot[sir_values < 0.0] = 0.0
    
    return pilot

In [ ]:
for i in range(0, number_of_samples):
    s, fs_s = sf.read(f'../data/train/s{i}.wav')
    y, fs_y = sf.read(f'../data/train/y{i}.wav')

    nr_of_mics = 9
    
    mix_signal = s + y
        
    X = STFT1024(mix_signal.T).transpose(0,2,1)
    SOI = STFT1024(s.T)
    INT = STFT1024(y.T)
    d, N, K = X.shape

    SIR_per_frames = calculate_SIR_for_frames(SOI, INT)

    SOI = SOI.transpose(0,2,1)
    INT = INT.transpose(0,2,1)

    pilot1 = create_pilot1(SIR_per_frames)
    SIR_pilot1[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot1)

    pilot2 = create_pilot2(SIR_per_frames)
    SIR_pilot2[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot2)

    pilot3 = create_pilot3(SIR_per_frames)
    SIR_pilot3[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot3)

    pilot4 = create_pilot4(SIR_per_frames)
    SIR_pilot4[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot4)

    SIR_df = pd.DataFrame({
        'SIR_pilot1': SIR_pilot1.flatten(),
        'SIR_pilot2': SIR_pilot2.flatten(),
        'SIR_pilot3': SIR_pilot3.flatten(),
        'SIR_pilot4': SIR_pilot4.flatten(),
    })
    SIR_df.to_csv('../results/SIR_pilots_comp.csv', index=True)

    print(i)


In [9]:
SIR_df = pd.DataFrame({
    'SIR_pilot1': SIR_pilot1.flatten(),
    'SIR_pilot2': SIR_pilot2.flatten(),
    'SIR_pilot3': SIR_pilot3.flatten(),
    'SIR_pilot4': SIR_pilot4.flatten(),
})
SIR_df.to_csv('../results/SIR_pilots_comp.csv', index=True)

In [4]:
SIR_df = pd.read_csv("../results/SIR_pilots_comp.csv")

In [8]:
df = pd.read_csv("../results/SIR_train.csv")

In [7]:
print(f"Průměrné zlepšení pilot 1 {np.mean(SIR_df["SIR_pilot1"][:number_of_samples] - df["SIR_original"][:number_of_samples])}")
print(f"Průměrné zlepšení pilot 2 {np.mean(SIR_df["SIR_pilot2"][:number_of_samples] - df["SIR_original"][:number_of_samples])}")
print(f"Průměrné zlepšení pilot 3 {np.mean(SIR_df["SIR_pilot3"][:number_of_samples] - df["SIR_original"][:number_of_samples])}")
print(f"Průměrné zlepšení pilot 4 {np.mean(SIR_df["SIR_pilot4"][:number_of_samples] - df["SIR_original"][:number_of_samples])}")

Průměrné zlepšení pilot 1 6.4106074708605805
Průměrné zlepšení pilot 2 6.410679781544097
Průměrné zlepšení pilot 3 6.41137451337926
Průměrné zlepšení pilot 4 6.411292031784456


In [ ]:


fig_box = go.Figure()

fig_box.add_trace(go.Box(
    y=df["imp_SIR_SOI_pilot"][:number_of_samples],
    name="Zlepšení SIR pro ideální pilot ze SOI"
))

fig_box.add_trace(go.Box(
    y=df["imp_SIR_ones_pilot"][:number_of_samples],
    name="Zlepšení SIR pro jednotkový pilot"
))

fig_box.add_trace(go.Box(
    y=df["imp_SIR_X_pilot"][:number_of_samples],
    name="Zlepšení SIR pro pilot z původního signálu"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot1"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Zlepšení SIR pro testovaný pilot 1"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot2"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Zlepšení SIR pro testovaný pilot 2"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot3"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Zlepšení SIR pro testovaný pilot 3"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot4"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Zlepšení SIR pro testovaný pilot 4"
))


fig_box.show()



In [18]:
fig_box = go.Figure()

fig_box.add_trace(go.Box(
    y=df["imp_SIR_SOI_pilot"][:number_of_samples],
    name="Pilot ze SOI"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot1"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 1"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot2"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 2"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot3"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 3"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot4"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 4"
))

fig_box.update_layout(
        title=dict(
        text="Zlepšení SIR extrahovaného signálu s testovanými ideálními piloty (trénovací data)",
        font=dict(size=18)
    ),
    xaxis=dict(
        title=dict(text="Typ pilotu", font=dict(size=18)),
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        title=dict(text="Zlepšení SIR [dB]", font=dict(size=18)),
        tickfont=dict(size=16)
    ),
    showlegend=False,
    height=800,
    width=800
)

fig_box.show()


In [11]:
fig_box = go.Figure()

fig_box.add_trace(go.Box(
    y=df["imp_SIR_SOI_pilot"][:number_of_samples],
    name="Pilot ze SOI"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot1"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 1"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot2"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 2"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot3"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 3"
))

fig_box.add_trace(go.Box(
    y=SIR_df["SIR_pilot4"][:number_of_samples] - df["SIR_original"][:number_of_samples],
    name="Pilot 4"
))

fig_box.add_trace(go.Box(
    y=df["imp_SIR_X_pilot"],
    name="Pilot z energie původního signálu"
))

fig_box.add_trace(go.Box(
    y=df["imp_SIR_ones_pilot"],
    name="Pilot z 1 (slepá extrakce)"
))

fig_box.update_layout(
        title=dict(
        text="Zlepšení SIR extrahovaného signálu s testovanými ideálními piloty (trénovací data)",
        font=dict(size=18)
    ),
    xaxis=dict(
        title=dict(text="Typ pilotu", font=dict(size=18)),
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        title=dict(text="Zlepšení SIR [dB]", font=dict(size=18)),
        tickfont=dict(size=16)
    ),
    showlegend=False,
    height=800,
    width=1000
)

fig_box.show()
